
1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

User Data Summary:
Income: 30000.0
Expenses: {'Rent': 12000.0, 'Groceries': 3000.0, 'Shopping': 4000.0, 'Daily_Spending': 2000.0, 'Loan': 0, 'Bills': 0, 'Others': 0}
Savings Goal: 2000 2300
Categories to Minimize: ['Groceries', 'Shopping', 'Daily_Spending']

No feasible solution found. Please check your constraints.


In [22]:
from pulp import *

def user_info():
    income = float(input("Enter your total income per month (In INR): "))
    expense_list = ["Rent", "Groceries", "Shopping", "Daily_Spending", "Loan", "Bills", "Others"]
    expense_dict = {k: 0 for k in expense_list}
    
    # Collect expenses from the user
    while True:
        print("\n1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit")
        choice = int(input("Select an expense to input or Exit: "))
        if choice == 8:
            break
        elif 1 <= choice <= 7:
            expense_name = expense_list[choice - 1]
            amount = float(input(f"Enter the estimated amount for {expense_name}: "))
            expense_dict[expense_name] = amount
        else:
            print("Invalid option. Try again.")
    
    savings_goal = float(input("Enter the amount you want to save this month: "))
    min_spendings_areas = [int(i) for i in input("Select categories to minimize expenses (e.g., '1 3 5'): ").split()]
    min_spendings = [expense_list[i - 1] for i in min_spendings_areas]
    
    print("\nUser Data Summary:")
    print("Income:", income)
    print("Expenses:", expense_dict)
    print("Savings Goal:", savings_goal)
    print("Categories to Minimize:", min_spendings)
    
    return income, expense_dict, min_spendings, savings_goal

def optimize_budget(income, expense_dict, min_spendings, savings_goal):
    """
    Optimize budget to maximize savings while respecting expense constraints
    
    Args:
        income: Total monthly income
        expense_dict: Dictionary of expense categories and their maximum amounts
        min_spendings: List of expense categories to minimize
        savings_goal: Minimum required savings
    
    Returns:
        dict: Optimized spending amounts for each category and savings
    """
    # Create the optimization problem
    prob = LpProblem("Budget_Optimization", LpMaximize)
    
    # Create decision variables for each expense category
    spending_vars = {}
    for category, max_amount in expense_dict.items():
        if max_amount > 0:  # Only create variables for categories with non-zero maximum
            spending_vars[category] = LpVariable(f"spend_{category}", 0, max_amount)
    
    # Create savings variable
    savings = LpVariable("savings", savings_goal, savings_goal+3000)
    
    # Objective: Maximize savings while minimizing selected categories
    # We'll subtract a small portion of the minimized categories from the objective
    optimization_expr = savings - 0.01 * lpSum(spending_vars[category] for category in min_spendings if category in spending_vars)
    prob += optimization_expr
    
    # Constraints
    
    # Total spending + savings must equal income
    prob += lpSum(spending_vars.values()) + savings == income
    
    # Minimum spending constraints (30% of maximum for essential categories)
    essential_categories = ["Rent", "Groceries", "Bills", "Loan"]
    for category in spending_vars:
        if category in essential_categories:
            min_amount = expense_dict[category] * 0.3  # 30% of maximum for essential categories
            prob += spending_vars[category] >= min_amount
        else:
            min_amount = expense_dict[category] * 0.1  # 10% of maximum for non-essential categories
            prob += spending_vars[category] >= min_amount
    
    # Solve the problem
    status = prob.solve(PULP_CBC_CMD(msg=False))
    
    if status != 1:
        return None
    
    # Get results
    results = {
        "savings": value(savings)
    }
    for category, var in spending_vars.items():
        results[category] = value(var)
    
    # Add zero values for categories that weren't included in optimization
    for category in expense_dict:
        if category not in results:
            results[category] = 0
    
    return results

def display_results(results, min_spendings):
    if results is None:
        print("\nNo feasible solution found. Please check your constraints.")
        return
        
    print("\nOptimized Budget:")
    print("-" * 50)
    
    # Print minimized categories first
    print("Categories to Minimize:")
    total_spent = 0
    for category in min_spendings:
        if category in results:
            amount = results[category]
            print(f"{category}: ₹{amount:,.2f}")
            total_spent += amount
    
    print("\nOther Categories:")
    # Print other categories
    for category, amount in results.items():
        if category != "savings" and category not in min_spendings:
            print(f"{category}: ₹{amount:,.2f}")
            total_spent += amount
    
    print("-" * 50)
    print(f"Total Spending: ₹{total_spent:,.2f}")
    print(f"Savings: ₹{results['savings']:,.2f}")
    print("-" * 50)
    print(f"Total: ₹{(total_spent + results['savings']):,.2f}")

def main():
    try:
        # Get user inputs
        income, expense_dict, min_spendings, savings_goal = user_info()
        
        # Validate inputs
        total_max_spending = sum(expense_dict.values())
        if total_max_spending + savings_goal > income:
            print("\nError: Your maximum spending limits plus minimum savings exceed your income.")
            print(f"Total maximum spending: ₹{total_max_spending:,.2f}")
            print(f"Minimum savings: ₹{savings_goal:,.2f}")
            print(f"Income: ₹{income:,.2f}")
            print(f"Deficit: ₹{(total_max_spending + savings_goal - income):,.2f}")
            return
        
        # Run optimization
        results = optimize_budget(income, expense_dict, min_spendings, savings_goal)
        
        # Display results
        display_results(results, min_spendings)
        
    except Exception as e:
        print(f"\nAn error occurred: {str(e)}")
        print("Please check your inputs and try again.")

if __name__ == "__main__":
    main()


1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

1. Rent 2. Groceries 3. Shopping  4. Daily Spending 5. Loan 6. Bills 7. Others 8. Exit

User Data Summary:
Income: 30000.0
Expenses: {'Rent': 12000.0, 'Groceries': 5000.0, 'Shopping': 4000.0, 'Daily_Spending': 2500.0, 'Loan': 2000.0, 'Bills': 2300.0, 'Others': 0}
Savings Goal: 2000.0
Categories to Minimize: ['Groceries', 'Shopping', 'Daily_Spending']

Optimized Budget:
--------------------------------------------------
Categories to Minimize:
Groceries: ₹5,000.00
Sho

In [10]:
import datetime
year = datetime.datetime.now().year
year

2024

In [18]:
print(datetime.date.today())

2024-10-30


In [20]:
import os
os.chdir("../")

(30.0, 20.0, 40.0, 30.0, 40.0, 30.0, 30.0, 20.0, 10.0)


In [61]:
columns = ["income","savings","rent","groceries","shopping","daily_spending","loan","bills","other_expenses"]
income = user[0]
optimal_budget = {}
for i in range(1,len(user)):
    optimal_budget[columns[i]] = user[i]
print(optimal_budget)

{'savings': 20.0, 'rent': 40.0, 'groceries': 30.0, 'shopping': 40.0, 'daily_spending': 30.0, 'loan': 30.0, 'bills': 20.0, 'other_expenses': 10.0}


In [53]:
columns = ["income","savings","rent","groceries","shopping","daily_spending","loan","bills","other_expenses"]
','.join(columns)

'income,savings,rent,groceries,shopping,daily_spending,loan,bills,other_expenses'

In [67]:
datetime.date.today()

datetime.date(2024, 10, 31)

In [1]:
from typing import Dict, List, Optional
from pulp import *
import logging

class BudgetOptimizer:
    def __init__(self):
        self.expense_categories = [
            "Rent", "Groceries", "Shopping", "Daily_spending",
            "Loan", "Bills", "Others"
        ]
        # Set up logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
    def validate_inputs(self, income: float, expense_dict: Dict[str, float], 
                       min_spendings: List[str], savings_goal: float) -> bool:
        """Validate input parameters before optimization."""
        try:
            # Check if income is positive
            if income <= 0:
                self.logger.error("Income must be positive")
                return False
                
            # Check if expense dictionary contains valid categories
            if not all(category in self.expense_categories for category in expense_dict):
                self.logger.error("Invalid expense categories found")
                return False
                
            # Check if minimum spending categories exist in expense dict
            if not all(category in expense_dict for category in min_spendings):
                self.logger.error("Minimum spending categories must exist in expense dictionary")
                return False
                
            # Check if savings goal + extra savings buffer is feasible
            total_min_expenses = sum(amount * 0.3 for category, amount in expense_dict.items() 
                                   if category in ["Rent", "Groceries", "Bills", "Loan"]) + \
                               sum(amount * 0.1 for category, amount in expense_dict.items() 
                                   if category not in ["Rent", "Groceries", "Bills", "Loan"])
            if total_min_expenses + savings_goal + 500 > income:
                self.logger.error("Savings goal plus buffer is not feasible with given constraints")
                return False
                
            return True
            
        except Exception as e:
            self.logger.error(f"Input validation failed: {str(e)}")
            return False

    def optimize_budget(
        self,
        income: float,
        expense_dict: Dict[str, float],
        min_spendings: List[str],
        savings_goal: float
    ) -> Optional[Dict[str, float]]:
        """
        Optimize budget to maximize savings while respecting expense constraints
        
        Args:
            income: Total monthly income
            expense_dict: Dictionary of expense categories and their maximum amounts
            min_spendings: List of expense categories to minimize
            savings_goal: Minimum required savings
        
        Returns:
            dict: Optimized spending amounts for each category and savings
        """
        try:
            # Validate inputs
            if not self.validate_inputs(income, expense_dict, min_spendings, savings_goal):
                return None

            # Create the optimization problem
            prob = LpProblem("Budget_Optimization", LpMinimize)
            
            # Create decision variables for each expense category
            spending_vars = {}
            for category, max_amount in expense_dict.items():
                if max_amount > 0:
                    spending_vars[category] = LpVariable(f"spend_{category}", 0, max_amount)
            
            # Create savings variable with upper bound of savings_goal + 500
            savings = LpVariable("savings", savings_goal, income)
            
            # Objective: Maximize savings while minimizing selected categories
            optimization_expr = savings - 0.01 * lpSum(
                spending_vars[category] for category in min_spendings if category in spending_vars
            )
            prob += optimization_expr
            
            # Constraint: Total spending + savings must equal income
            prob += lpSum(spending_vars.values()) + savings == income, "Income_Balance"
            
            # Minimum spending constraints
            essential_categories = ["Rent", "Groceries", "Bills", "Loan"]
            for category in spending_vars:
                if category in essential_categories:
                    min_amount = expense_dict[category] * 0.3  # 30% of maximum for essential
                    prob += spending_vars[category] >= min_amount, f"Min_{category}"
                else:
                    min_amount = expense_dict[category] * 0.1  # 10% of maximum for non-essential
                    prob += spending_vars[category] >= min_amount, f"Min_{category}"
            
            # Solve the problem
            solver = PULP_CBC_CMD(msg=False)
            status = prob.solve(solver)
            
            if status != 1:
                self.logger.error(f"Optimization failed with status: {status}")
                return None
            
            # Get results
            results = {"Savings": value(savings)}
            for category, var in spending_vars.items():
                results[category] = round(value(var), 2)
            
            # Add zero values for categories that weren't included
            for category in expense_dict:
                if category not in results:
                    results[category] = 0
            
            self.logger.info("Budget optimization completed successfully")
            return results
            
        except Exception as e:
            self.logger.error(f"Budget optimization failed: {str(e)}")
            return None

In [15]:
# from utils.optimal_budget_utils import BudgetOptimizer
income = 40000
expenses_dict = {
    "Rent":30000,
    "Groceries": 6000,
    "Shopping": 1000,
    "Daily_spending": 1800,
    "Loan": 1000,
    "Bills": 600,
    "Others":0
}
savings = 1000
min_spendings = ["Groceries", "Shopping", "Daily_spending"]
bo = BudgetOptimizer()
results = bo.optimize_budget(income=income,expense_dict=expenses_dict,min_spendings=min_spendings,savings_goal=savings)
print(results)

{'Savings': 1000.0, 'Rent': 30000.0, 'Groceries': 5000.0, 'Shopping': 600.0, 'Daily_spending': 1800.0, 'Loan': 1000.0, 'Bills': 600.0, 'Others': 0}


In [47]:

class BudgetOptimizer:
    def __init__(self):
        self.expense_categories = [
            "Rent", "Groceries", "Shopping", "Daily_spending",
            "Loan", "Bills", "Others"
        ]
        
    def optimize_budget(
        self,
        income: float,
        expense_dict: Dict[str, float],
        min_spendings: List[str],
        savings_goal: float
    ) -> Optional[Dict[str, float]]:
        """
        Optimize budget to maximize savings while respecting expense constraints
        
        Args:
            income: Total monthly income
            expense_dict: Dictionary of expense categories and their maximum amounts
            min_spendings: List of expense categories to minimize
            savings_goal: Minimum required savings
        
        Returns:
            dict: Optimized spending amounts for each category and savings
        """
        # Create the optimization problem
        prob = LpProblem("Budget_Optimization", LpMaximize)
        
        # Create decision variables for each expense category
        spending_vars = {}
        for category, max_amount in expense_dict.items():
            if max_amount > 0:
                spending_vars[category] = LpVariable(f"spend_{category}", 0, max_amount)
        
        # Create savings variable
        savings = LpVariable("savings", savings_goal)  # Allow for higher savings
        
        # Objective: Maximize savings while minimizing selected categories
        optimization_expr = savings - 0.01 * lpSum(
            spending_vars[category] for category in min_spendings if category in spending_vars
        )
        prob += optimization_expr
        
        # Constraint: Total spending + savings must equal income
        prob += lpSum(spending_vars.values()) + savings == income
        
        # Minimum spending constraints
        essential_categories = ["Rent", "Groceries", "Bills", "Loan"]
        for category in spending_vars:
            if category in essential_categories:
                min_amount = expense_dict[category]   # 30% of maximum for essential
                prob += spending_vars[category] >= min_amount
            else:
                min_amount = expense_dict[category] *0.5  # 10% of maximum for non-essential
                prob += spending_vars[category] >= min_amount
        
        # Solve the problem
        status = prob.solve(PULP_CBC_CMD(msg=False))
        
        if status != 1:
            return None
        
        # Get results
        results = {"Savings": value(savings)}
        for category, var in spending_vars.items():
            results[category] = round(value(var), 2)
        
        # Add zero values for categories that weren't included
        for category in expense_dict:
            if category not in results:
                results[category] = 0
                
        return results

In [48]:
# from utils.optimal_budget_utils import BudgetOptimizer
income = 40000
expenses_dict = {
    "Rent":23200,
    "Groceries": 5000,
    "Shopping": 4000,
    "Daily_spending": 1800,
    "Loan": 2900,
    "Bills": 2000,
    "Others":1100
}
savings = 1000
min_spendings = ["Others", "Shopping", "Daily_spending"]
bo = BudgetOptimizer()
results = bo.optimize_budget(income=income,expense_dict=expenses_dict,min_spendings=min_spendings,savings_goal=savings)
print(results)

{'Savings': 3450.0, 'Rent': 23200.0, 'Groceries': 5000.0, 'Shopping': 2000.0, 'Daily_spending': 900.0, 'Loan': 2900.0, 'Bills': 2000.0, 'Others': 550.0}


In [50]:
dict1 = {'x1':2,'x2':3}
dict2 = {'x1':1,'x2':2}
for i in dict1:
    dict1[i]+=dict2[i]

In [51]:
dict1

{'x1': 3, 'x2': 5}

In [3]:
import os 
os.chdir('../')

[(11400.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2024, 11)]